In [2]:
! chmod 600 /content/kaggle.json

In [3]:
! KAGGLE_CONFIG_DIR=/content/ kaggle datasets download -d vuppalaadithyasairam/bone-fracture-detection-using-xrays

 89% 153M/172M [00:01<00:00, 87.4MB/s]
100% 172M/172M [00:01<00:00, 110MB/s] 


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [5]:
import zipfile
zip_file=zipfile.ZipFile('/content/bone-fracture-detection-using-xrays.zip','r')
zip_file.extractall('/tmp/')

In [6]:
import os
for dirname, _, filenames in os.walk('/tmp/archive (6)'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Streaming output truncated to the last 5000 lines.
/tmp/archive (6)/train/fractured/54-rotated3-rotated3-rotated1.jpg
/tmp/archive (6)/train/fractured/51-rotated3-rotated1-rotated1.jpg
/tmp/archive (6)/train/fractured/81-rotated3-rotated1-rotated1.jpg
/tmp/archive (6)/train/fractured/83-rotated1-rotated2-rotated1.jpg
/tmp/archive (6)/train/fractured/16-rotated1-rotated3-rotated3.jpg
/tmp/archive (6)/train/fractured/84-rotated1-rotated1-rotated3.jpg
/tmp/archive (6)/train/fractured/45-rotated3-rotated3.jpg
/tmp/archive (6)/train/fractured/44-rotated3-rotated2-rotated2.jpg
/tmp/archive (6)/train/fractured/16-rotated3-rotated2.jpg
/tmp/archive (6)/train/fractured/62.jpg
/tmp/archive (6)/train/fractured/119-rotated2-rotated1.jpg
/tmp/archive (6)/train/fractured/26-rotated3.jpg
/tmp/archive (6)/train/fractured/117-rotated2-rotated2-rotated1.jpg
/tmp/archive (6)/train/fractured/107-rotated3-rotated3-rotated1.jpg
/tmp/archive (6)/train/fractured/83-rotated1-rotated2-rotated3.jpg
/tmp/archive 

In [7]:
train_path= '/tmp/archive (6)/train'
test_path='/tmp/archive (6)/val'

In [8]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(rescale=1./255)

val_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [9]:
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')
validation_generator = val_datagen.flow_from_directory(
    test_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

Found 8863 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [10]:
# Membangun model CNN
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [11]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [12]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

lrp = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=2)

checkpoint = ModelCheckpoint(filepath='model_checkpoint.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max', save_weights_only=True)
callbacks = [checkpoint, lrp]

model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)


Epoch 1/10
276/276 [==============================] - 470s 2s/step - loss: 0.6572 - accuracy: 0.5888 - val_loss: 0.5855 - val_accuracy: 0.7604
Epoch 2/10
276/276 [==============================] - 466s 2s/step - loss: 0.5901 - accuracy: 0.6609 - val_loss: 0.5833 - val_accuracy: 0.6163
Epoch 3/10
276/276 [==============================] - 470s 2s/step - loss: 0.4765 - accuracy: 0.7551 - val_loss: 0.6527 - val_accuracy: 0.6337
Epoch 4/10
276/276 [==============================] - 458s 2s/step - loss: 0.3928 - accuracy: 0.8126 - val_loss: 0.6897 - val_accuracy: 0.6042
Epoch 5/10
276/276 [==============================] - 462s 2s/step - loss: 0.3197 - accuracy: 0.8546 - val_loss: 0.5783 - val_accuracy: 0.6892
Epoch 6/10
276/276 [==============================] - 463s 2s/step - loss: 0.2717 - accuracy: 0.8796 - val_loss: 0.5461 - val_accuracy: 0.7396
Epoch 7/10
276/276 [==============================] - 459s 2s/step - loss: 0.2305 - accuracy: 0.9028 - val_loss: 0.4297 - val_accuracy: 0.7917

In [13]:
model.evaluate(train_generator)

277/277 [==============================] - 160s 578ms/step - loss: 0.0873 - accuracy: 0.9719


[0.08734571933746338, 0.9719056487083435]

In [14]:
model.evaluate(validation_generator)

19/19 [==============================] - 11s 553ms/step - loss: 0.6271 - accuracy: 0.7533


[0.6270520091056824, 0.753333330154419]

In [15]:
from tensorflow.keras.utils import load_img, img_to_array
img = load_img('../input/bone-fracture-detection-using-xrays/archive (6)/val/not fractured/2-rotated1.jpg',target_size=(224,224))
imag = img_to_array(img)
imaga = np.expand_dims(imag,axis=0) 
ypred = model.predict(imaga)
print(ypred)
a=ypred[0]
if a<0.5:
      op="Fracture"   
else:
      op="Normal"
plt.imshow(img)
print("THE UPLOADED X-RAY IMAGE IS: "+str(op))

FileNotFoundError: ignored